In [1]:
from flask import Flask, request, jsonify, render_template
import random
import tensorflow
from tensorflow.keras.models import load_model
import numpy as np
import librosa
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import os
import speech_recognition as sr

In [2]:
app = Flask(__name__, template_folder='template', static_folder='static')
model=load_model(r'EmotionEmergency.h5')

In [3]:
@app.route('/')
def home():
    return render_template('index.html')

In [4]:
from unittest import result


@app.route('/predict_',methods=['GET','POST'])
def predict_():

    # get file from POST request and save it
    # audio_file = request.files["file"]
    # file_name = str(random.randint(0, 100000))
    # audio_file.save(file_name)
    audio_file = r"C:\Users\saxen\Downloads\output.wav"
    # result_list=[]
    # count_list=[]
    # livepredictions=""
    try:
        r = sr.Recognizer()
        # open the file
        with sr.AudioFile(audio_file) as source:
        # listen for the data (load audio to memory)
            # r.adjust_for_ambient_noise(source)
            audio_data = r.listen(source)
        # recognize (convert from speech to text)
            text = r.recognize_google(audio_data)
            print(text.casefold())
            speech_text= text.casefold()
    except sr.UnknownValueError:
        # result_list =["Blank Call", "", ""]
        # count_list=["", "", ""]
        # df = pd.DataFrame(zip(result_list,count_list))
        # livepredictions="Blank Call"
    # if livepredictions == "Blank Call":
        os.remove(audio_file)
        return render_template('result.html', prediction_text='{}'.format("Blank Call"), text_emotion1='{}'.format("Blank Call"),text_emotion2='{}'.format(""),text_emotion3='{}'.format(""))
    if speech_text!="":
        Features = pd.read_csv(r'D:\SER\E2\features_Emergency.csv')
        Y = Features['labels'].values
        encoder = OneHotEncoder()
        Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()
        
        # NOISE
        def noise(data):
            noise_amp = 0.035*np.random.uniform()*np.amax(data)
            data = data + noise_amp*np.random.normal(size=data.shape[0])
            return data
        # # STRETCH
        # def stretch(data, rate=0.8):
        #     return librosa.effects.time_stretch(data, rate)
        # # PITCH
        # def pitch(data, sampling_rate, pitch_factor=0.7):
        #     return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

        def feat_ext(data):
            mfcc = np.mean(librosa.feature.mfcc(y=data, sr=22050).T, axis=0)
            return mfcc

        def get_feat(path):
            data, sample_rate = librosa.load(path, duration=5, offset=0.6)
            # normal data
            res1 = feat_ext(data)
            result = np.array(res1)
            #data with noise
            noise_data = noise(data)
            res2 = feat_ext(noise_data)
            result = np.vstack((result, res2))
            return result

        try:
            feature = get_feat(audio_file)
            test =np.expand_dims(feature, axis=2)
            livepreds = model.predict(test)
            livepredictions = (encoder.inverse_transform((livepreds)))
        except ValueError:
            livepredictions = ['Blank Call']

        dataset = pd.read_csv(r'WORDS_EMOTIONS.csv')
        # dataset = dataset.decode('ISO-8859-1', 'ignore')
        dataset.astype({'Phrases':'string', 'Emotions':'string'}).dtypes
        X = dataset['Phrases']
        y = dataset['Emotions']
        emotion_list=[]
        for i in range(len(X)):
            if X[i] in speech_text:
                emotion_list.append(y[i])  
        
        # if len(emotion_list)>0:
        try:
            count_angry=emotion_list.count("abusive")
            count_notprank=emotion_list.count("Not_Prank")
            count_prank=emotion_list.count("prank")
            total = count_prank+count_angry+count_notprank
            angry_percentage = count_angry/total*100
            prank_percentage = count_prank/total*100
            notprank_percentage = count_notprank/total*100
            count_list = [angry_percentage,notprank_percentage ,prank_percentage]
            for ind, i in enumerate(count_list):
                count_list[ind] = "{}%".format(i)
            result_list = ['Abusive','Not Prank','Prank']
            df = pd.DataFrame(zip(result_list,count_list))
        except ZeroDivisionError:
            count_list = ['0%','0%','0%']
            # for ind, i in enumerate(count_list):
                # [ind] = "{}%".format(i)
            result_list = ['Abusive','Not Prank','Prank']
            df = pd.DataFrame(zip(result_list,count_list))

        
        os.remove(audio_file)


        # return render_template('result.html', prediction_text='{}'.format(livepredictions[0]))
        return render_template('result.html', prediction_text='{}'.format(livepredictions[0]), text_emotion1='{}'.format(df.iloc[0,:].to_string(index=False,header=False)),text_emotion2='{}'.format(df.iloc[1,:].to_string(index=False,header=False)),text_emotion3='{}'.format(df.iloc[2,:].to_string(index=False,header=False)))


In [5]:
# @app.route('/results',methods=['POST'])
# def results():
#     data = request.get_json(force=True)
#     prediction = model.predict(data.values())
    
#     return jsonify(prediction)

In [6]:
if __name__ == "__main__":
    app.run(debug=True,use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Aug/2022 19:19:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 19:19:52] "GET /static/images/footer_logo.PNG HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:19:52] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:19:52] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:19:52] "GET /static/js/main.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:19:52] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Aug/2022 19:53:00] "GET /predict_ HTTP/1.1" 500 -
Traceback (most recent call last):
  File "c:\Users\saxen\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2548, in __call__
    return self.wsgi_app(environ, start_response)
  File "c:\Users\saxen\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2528, in wsgi_app
    response = self.handle_exception(e)
  File "c:\Users\saxen\AppData\Local\Programs\Python\

column and limbs as soon as possible
1/1 [==============================] - 2s 2s/step


127.0.0.1 - - [25/Aug/2022 19:53:26] "GET /predict_ HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 19:53:26] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:26] "GET /static/js/main.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:26] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:26] "GET /static/images/footer_logo.PNG HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 19:53:36] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:36] "GET /static/js/main.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:36] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:36] "GET /static/images/footer_logo.PNG HTTP/1.1" 304 -


kollam ambulance
1/1 [==============================] - 0s 42ms/step


127.0.0.1 - - [25/Aug/2022 19:53:51] "GET /predict_ HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 19:53:51] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:51] "GET /static/js/main.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:51] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:51] "GET /static/images/footer_logo.PNG HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 19:53:56] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:56] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:56] "GET /static/js/main.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:53:56] "GET /static/images/footer_logo.PNG HTTP/1.1" 304 -


call an ambulance as soon as possible
1/1 [==============================] - 0s 38ms/step


127.0.0.1 - - [25/Aug/2022 19:54:11] "GET /predict_ HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 19:54:11] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:54:11] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:54:11] "GET /static/js/main.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:54:11] "GET /static/images/footer_logo.PNG HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:54:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 19:54:15] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:54:15] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:54:15] "GET /static/js/main.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 19:54:15] "GET /static/images/footer_logo.PNG HTTP/1.1" 304 -


can you call an ambulance as soon as possible
1/1 [==============================] - 0s 30ms/step


127.0.0.1 - - [25/Aug/2022 20:03:34] "GET /predict_ HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 20:03:34] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 20:03:34] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 20:03:34] "GET /static/js/main.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 20:03:34] "GET /static/images/footer_logo.PNG HTTP/1.1" 304 -
